# [프로젝트] 복잡한 자연어 쿼리를 처리하는 Text-to-SQL   

Text-to-SQL 문제는 LLM의 대표적인 활용 분야 중 하나입니다.   
LangChain의 기능을 활용하여, 간단하게 SQL 정보를 전달하고   
 쿼리를 실행, 생성하는 기능을 만들어 보겠습니다.

이번 실습에서는 다음의 DB를 사용합니다.   

easy_sql.db : 도서관 DB  
intermediate_sql.db : 병원 DB   
hard_sql.db : 대학교 DB


In [ ]:
!pip install langgraph langchain langchain_google_genai google_generativeai langchain_community python-dotenv -q

**세션 다시 시작** 해 주세요!

In [ ]:
import zipfile

with zipfile.ZipFile('sql.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

## LangChain의 Utilities 활용하기

SQLite3을 이용해 db에 접근한 뒤, DB 정보를 불러옵니다.

In [ ]:
import sqlite3
from langchain_community.utilities import SQLDatabase


db = SQLDatabase.from_uri("sqlite:///easy_sql.db")


get_table_info()를 통해 스키마를 가져옵니다.    
테이블의 목록 + 3개 Row를 가져오는 작업을 기본으로 수행합니다.

In [ ]:
db_schema = db.get_table_info()
print(db_schema)

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# 환경 변수 로드
load_dotenv()

os.environ['LANGCHAIN_PROJECT'] = 'LangGraph_FastCampus'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_TRACING_V2']='true'

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

구조화된 출력을 구성합니다.

In [ ]:
from pydantic import BaseModel, Field
import sqlite3

class SQLQuery(BaseModel):
    """SQL 쿼리"""
    explanation: str = Field(description="쿼리의 목적과 기대되는 결과 설명")
    query: str = Field(description="생성된 SQL 쿼리 (반드시 유효한 SQL 문법을 사용)")


In [ ]:
from langchain.prompts import ChatPromptTemplate

structured_llm = llm.with_structured_output(SQLQuery)
prompt = ChatPromptTemplate(["""
당신은 SQL 전문가입니다. 다음 데이터베이스 스키마와 사용자 질의를 분석하여 SQL 쿼리를 작성하세요.

## 데이터베이스 스키마:
{db_schema}

## 사용자 질의:
{query}


테이블을 참고하여, 질문을 쿼리로 변환하세요.
SQLite의 경우 LIMIT clause 를 사용하면 됩니다.
데이터의 결과를 더 잘 전달하기 위해 정렬할 수 있습니다.
주어진 예시 row에 없는 내용이라도, 쿼리를 작성하세요.

정답과 무관한 컬럼에 대한 쿼리를 수행하지 말고, 필요한 내용만 추출하세요.
아래에 주어지는 컬럼 이름과 테이블 정보를 꼭 참고하세요.
존재하지 않는 컬럼에 대한 쿼리를 하지 않도록 주의하세요.
만약 '오늘'과 같이 현재 날짜에 대한 질문이 들어오면 date() 등을 사용하세요.
컬럼 이름은 "로 구분하고, 전체 출력은 마크다운으로 수행하세요.
"""])


sample = {'db_schema':db_schema, 'query':'책이 총 몇 권 있어요?'}

chain = prompt | structured_llm
query = chain.invoke(sample)

query


생성한 쿼리는 run()으로 실행합니다.

In [ ]:
db.run(query.query)

전체 기능을 노드로 연결합니다.

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    db_schema: str
    db_path: str
    query: str
    result: str
    sql_query : str
    final_analysis: str

def generate_query(state: State):
    structured_llm = llm.with_structured_output(SQLQuery)
    prompt = ChatPromptTemplate(["""
    당신은 SQL 전문가입니다. 다음 데이터베이스 스키마와 사용자 질의를 분석하여 SQL 쿼리를 작성하세요.

    ## 데이터베이스 스키마:
    {db_schema}

    ## 사용자 질의:
    {query}


    테이블을 참고하여, 질문을 쿼리로 변환하세요.
    SQLite의 경우 LIMIT clause 를 사용하면 됩니다.
    데이터의 결과를 더 잘 전달하기 위해 정렬할 수 있습니다.
    주어진 예시 row에 없는 내용이라도, 쿼리를 작성하세요.

    정답과 무관한 컬럼에 대한 쿼리를 수행하지 말고, 필요한 내용만 추출하세요.
    아래에 주어지는 컬럼 이름과 테이블 정보를 꼭 참고하세요.
    존재하지 않는 컬럼에 대한 쿼리를 하지 않도록 주의하세요.
    만약 '오늘'과 같이 현재 날짜에 대한 질문이 들어오면 date() 등을 사용하세요.
    컬럼 이름은 "로 구분하고, 전체 출력은 마크다운으로 수행하세요.
    """])
    chain = prompt | structured_llm
    sql_query = chain.invoke(state).query
    state['sql_query'] = sql_query
    return state

def execute_query(state: State):

    sql_query = state['sql_query']
    db_path = state['db_path']
    db = SQLDatabase.from_uri(db_path)

    result = db.run(state['sql_query'])

    state['result'] = result
    return state

def explain_result(state: State):
    prompt = f"""
    당신은 데이터 분석 전문가입니다. SQL 쿼리 결과를 분석하여 사용자가 이해하기 쉽게 설명해주세요.

    ## 원래 질문:
    {state["query"]}

    ## 실행된 SQL 쿼리:
    {state["sql_query"]}

    ## 쿼리 결과:
    {state["result"]}

    ## 작업:
    1. 원래 질문에 직접적으로 답변하세요.
    2. 결과에서 발견된 주요 인사이트를 추출하세요.
    3. 데이터를 종합적으로 분석하고 명확하게 설명하세요.
    4. 결과가 원래 질문의 모든 측면을 다루는지 확인하세요.
    """
    response = llm.invoke(prompt)
    state["final_analysis"] = response.content
    return state

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("generate_query", generate_query)
builder.add_node("execute_query", execute_query)
builder.add_node("explain_result", explain_result)

# 노드 연결
builder.add_edge("generate_query", "execute_query")
builder.add_edge("execute_query", "explain_result")
builder.add_edge("explain_result", END)

# 시작 노드 설정
builder.add_edge(START, "generate_query")

# 그래프 컴파일
graph = builder.compile()
graph


In [ ]:
db_path = "sqlite:///easy_sql.db"
# 데이터베이스 스키마 미리 가져오기
db = SQLDatabase.from_uri(db_path)
db_schema = db.get_table_info()

test_queries=[
    "가장 많은 매출을 올린 작가의 모든 도서 목록과 재고 현황을 조회해주세요.",
    "IT 분야 도서 목록 알려주세요.",
    "리뷰가 좋은 책들 추천해주세요.",
]

for test_query in test_queries:

    initial_state = {
        'query':test_query,
        "db_path": db_path,
        "db_schema": db_schema  # 미리 가져온 스키마 정보 설정
    }
    result = graph.invoke(initial_state)

    print('기존 질문:', test_query)
    print('최종 분석 결과:', result["final_analysis"])

    print('-------------')

